# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client("sagemaker")

## You must have already run a hyperparameter tuning job to analyze it here.
## The Hyperparameter tuning jobs you have run are listed in the Training section on your SageMaker dashboard.
## Copy the name of a completed job you want to analyze from that list.
## For example: tuning_job_name = 'mxnet-training-201007-0054'.
#tuning_job_name = "YOUR-HYPERPARAMETER-TUNING-JOB-NAME"
tuning_job_name = "xgboost-210726-1454"

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [5]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"] != "Maximize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["MetricName"]

20 training jobs have completed


In [6]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 7, 26, 14, 54, 17, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7764379978179932},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 7, 26, 14, 58, 11, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:168402291854:training-job/xgboost-210726-1454-003-cd9f801e',
 'TrainingJobName': 'xgboost-210726-1454-003-cd9f801e',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 7, 26, 14, 56, 55, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.9120572195722887',
                          'eta': '0.11097433466769813',
                          'max_depth': '3',
                          'min_child_weight': '8.982268946607327'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [7]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.7060909867286682, 'highest': 0.7764379978179932}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
18,1.912057,0.110974,3.0,8.982269,xgboost-210726-1454-003-cd9f801e,Completed,0.776438,2021-07-26 14:56:55+00:00,2021-07-26 14:58:11+00:00,76.0
14,1.558127,0.096159,5.0,7.278450,xgboost-210726-1454-007-4cae7d77,Completed,0.776127,2021-07-26 15:05:34+00:00,2021-07-26 15:06:44+00:00,70.0
8,0.024231,0.092863,6.0,9.270447,xgboost-210726-1454-013-9b30f554,Completed,0.775667,2021-07-26 15:13:48+00:00,2021-07-26 15:14:43+00:00,55.0
1,1.345908,0.069151,4.0,7.654083,xgboost-210726-1454-020-ec7108cc,Completed,0.775541,2021-07-26 15:23:33+00:00,2021-07-26 15:24:41+00:00,68.0
2,0.024231,0.112863,6.0,9.450447,xgboost-210726-1454-019-236e0bf2,Completed,0.775080,2021-07-26 15:23:24+00:00,2021-07-26 15:24:42+00:00,78.0
9,0.038330,0.075987,4.0,10.000000,xgboost-210726-1454-012-350f451a,Completed,0.774814,2021-07-26 15:11:49+00:00,2021-07-26 15:12:39+00:00,50.0
0,0.842551,0.035227,8.0,7.951912,xgboost-210726-1454-021-c1851e4e,Completed,0.773737,2021-07-26 15:29:11+00:00,2021-07-26 15:30:29+00:00,78.0
4,1.997059,0.198515,6.0,9.249946,xgboost-210726-1454-017-c357b7c7,Completed,0.771896,2021-07-26 15:19:29+00:00,2021-07-26 15:20:43+00:00,74.0
5,1.919577,0.025786,5.0,8.376533,xgboost-210726-1454-016-11b08bff,Completed,0.770500,2021-07-26 15:19:25+00:00,2021-07-26 15:20:14+00:00,49.0
17,1.825195,0.091353,2.0,9.540260,xgboost-210726-1454-004-a20d82de,Completed,0.769774,2021-07-26 15:02:01+00:00,2021-07-26 15:03:12+00:00,71.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [10]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper():
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [12]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))